In [50]:
import snap 
import csv
import pandas as pd
import random
import networkx as nx
from node2vec import Node2Vec
#from deepwalk import DeepWalk
#from graphembedding.sdne import SDNE
#from gem.embedding.hope import HOPE
from gensim.models import Word2Vec

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import joblib
from sklearn.model_selection import ParameterGrid

from scipy.sparse import coo_matrix


%run Fairness.ipynb
%run Embeddings_Classification.ipynb

Create network based on the file

In [2]:
path_pokec_dataset = 'Datasets/soc-pokec-relationships.txt'
graph = snap.TNEANet.New()

file = open(path_pokec_dataset)
file_content = file.read()


In [3]:
G = snap.LoadEdgeList(snap.TNEANet, path_pokec_dataset, 0, 1, '\t')

In [4]:
nodes = G.GetNodes()
print(nodes)
print(G.GetEdges())

1632803
30622564


In [5]:
list_nodes = []
for nodes in G.Nodes():
    list_nodes.append(nodes.GetId())
print(len(list_nodes))

1632803


Peform random sample to create a network with 1000 edges

In [6]:
random.seed(42)

# Perform random sampling
sampled_nodes = random.sample(list_nodes, 1000)


In [7]:
print(len(sampled_nodes))
print(sampled_nodes[3])

1000
1308051


In [8]:
sampled_graph =snap.TNEANet.New()

# Add nodes to the sampled graph
for node_id in sampled_nodes:
    sampled_graph.AddNode(node_id)

In [9]:
for node in sampled_graph.Nodes():
    node_x_neighbors = set() 

    for neighbor in G.GetNI(node.GetId()).GetOutEdges():
        node_x_neighbors.add(neighbor)
    for i in node_x_neighbors:
        if node_id != i and sampled_graph.IsNode(i):
                sampled_graph.AddEdge(node_id, i)


In [10]:
print(sampled_graph.GetNodes())
print(sampled_graph.GetEdges())
print(sampled_graph.IsConnected())

1000
14
False


Find the largest connected component and create a subgaph based on that

In [11]:
newGraph = sampled_graph.GetMxWcc()
print(newGraph.GetNodes())
print(newGraph.GetEdges())
print(newGraph.IsConnected())

15
14
True


create 2 dictionaries one for the attribute (gender) and one for the target (public account or not)

In [12]:
list_attributes = {}
public = {}
with open('Datasets/soc-pokec-profiles.txt', 'r', encoding='utf-8') as csv_file:
    lines = csv_file.readlines()
    for i in range(0, len(lines)):
        #print(lines[i])
        modified_string = lines[i].replace('\t', ' ')
        modified_string = modified_string.split()
        public.update({int(modified_string[0]): modified_string[1]})
        #print(modified_string[0], modified_string[3])
        list_attributes.update({int(modified_string[0]): modified_string[3]})
#print(list_attributes)

In [13]:
print(len(public))

1632803


In [14]:
print(len(list_attributes))

1632803


In [15]:
sorted_public = dict(sorted(public.items()))

In [16]:
print(len(sorted_public))

1632803


In [17]:
#sorrt the dictionarries by the keys in ascending order
sorted_list_attributes = dict(sorted(list_attributes.items()))



In [18]:
newPublic = {}
newGender = {}
for nodes in newGraph.Nodes():
    id_nodes = nodes.GetId()
    values = sorted_list_attributes[id_nodes]
    values_publ = sorted_public[id_nodes]
    if(newGraph.IsNode(id_nodes)):
        newGraph.AddIntAttrDatN(int(id_nodes), int(values), "gender")
        newGender.update({int(id_nodes): int(values)})
        #print('Att')
    if(newGraph.IsNode(id_nodes)):
        #print('keysP')
        newPublic.update({id_nodes: values_publ})
print(len(newPublic))
print(len(newGender))

    

15
15


In [19]:
for node in newGraph.Nodes():
    print(newGraph.GetIntAttrDatN(node.GetId(), "gender"))

0
1
0
0
1
1
1
0
0
1
1
0
0
1
1


In [20]:
print(len(newPublic))

15


In [21]:
print('hello')

hello


Transform the Snap Graph to a NetworkX

In [22]:
def tansformGraph_toNetwork(G_snap):
    # Convert the snap.py graph to a NetworkX graph
    G_nx = nx.Graph()
    for node in G_snap.Nodes():
        G_nx.add_node(node.GetId())
        nx.set_node_attributes(G_nx, list_attributes, "Gender")
    for edge in G_snap.Edges():
        G_nx.add_edge(edge.GetSrcNId(), edge.GetDstNId())
    
    return G_nx

In [23]:
nx_Graph = tansformGraph_toNetwork(newGraph)

print(nx_Graph.number_of_nodes())
print(nx_Graph.number_of_edges())
print(type(nx_Graph))

15
14
<class 'networkx.classes.graph.Graph'>


In [31]:
# Resetting the index of the nodes from 0 to number of nodes for SINE and AE
new_indices = range(0, nx_Graph.number_of_nodes())
mapping = {old_index: new_index for new_index, old_index in zip(new_indices, nx_Graph.nodes())}
x_Graph = nx.relabel_nodes(nx_Graph, mapping)

In [32]:
print(type(x_Graph))

<class 'networkx.classes.graph.Graph'>


In [33]:
newList = list(newGender.values())

print(type(newList[0]))

<class 'int'>


In [34]:
lenvalues = len(newGender.values())
print(lenvalues)
datF = pd.DataFrame(newGender.values())


15


In [35]:
datF.head()

,0
0,0
1,1
2,0
3,0
4,1


In [51]:
df_features = datF.apply(pd.to_numeric, errors='coerce')
row_indices, col_indices = np.where(~np.isnan(datF.values))
print(row_indices)
print(col_indices)
print(np.array(list(newGender.values())))

# Create the COO sparse matrix
coo_matrix = coo_matrix((datF.values.ravel(), (row_indices, col_indices)), shape=(datF.shape[0], datF.shape[1]))
print(coo_matrix)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 1 1 1 0 0 1 1 0 0 1 1]
  (0, 0)	0
  (1, 0)	1
  (2, 0)	0
  (3, 0)	0
  (4, 0)	1
  (5, 0)	1
  (6, 0)	1
  (7, 0)	0
  (8, 0)	0
  (9, 0)	1
  (10, 0)	1
  (11, 0)	0
  (12, 0)	0
  (13, 0)	1
  (14, 0)	1


In [52]:
#Parameter
parameter_grid = {
    'dimensions': [16, 32],
    'walk_length': [20, 30],
    'walk_number': [25, 75],
    'workers': [2, 4]
}
print(type(nx_Graph))

#apply SINE embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_w,best_fpr_m, dataframe = grid_search_SINE(x_Graph, coo_matrix, parameter_grid,  list(newPublic.values()), list(newGender.values()))

<class 'networkx.classes.graph.Graph'>
done create embeding vectors
['1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '0' '1']
['0', '1']
predic
['1', '1']
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
done iteration
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
PPV_f and PPV_m is equal: False
['0', '1']
predic
['1', '1']
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
fpF: 1.0
fpM: 1.0
done create embeding vectors
['1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '0' '1']
['0', '1']
predic
['1', '1']
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
done iteration
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
PPV_f and PPV_m is equal: False
['0', '1']
predic
['1', '1']
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
fpF: 1.0
fpM: 1.0
done create embeding vectors
['1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '0' '1']
['0', '1']
predic
['1', '1']
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
done iteration
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
PPV_f and PPV_m is equal: False
['0', 

In [53]:
print(best_params, best_fpr_w,best_fpr_m, dataframe)

{'dimensions': 16, 'walk_length': 20, 'walk_number': 25, 'workers': 2} 1.0 1.0                            Female      Male
Group Fairness                0.0  0.000000
Predictive Parity             0.5  0.666667
False Positive Error Rate     1.0  1.000000
Equalized Opportunity         0.0  0.000000
Equalized Odds PPV            1.0  1.000000
Equalized Odds FPR            1.0  1.000000


In [54]:
# Parameter
parameter_grid = {
    'dimensions': [16, 32],
    'walk_length': [20, 30],
    'walk_number': [25, 75],
    'workers': [2, 4]
}

#apply AE embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_w,best_fpr_m, dataframe = grid_search_AE(x_Graph, coo_matrix, parameter_grid,  list(newPublic.values()), list(newGender.values()))

done create embeding vectors
['1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '0' '1']
['0', '1']
predic
['1', '1']
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
done iteration
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
PPV_f and PPV_m is equal: False
['0', '1']
predic
['1', '1']
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
fpF: 1.0
fpM: 1.0
done create embeding vectors
['1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '0' '1']
['0', '1']
predic
['1', '1']
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
done iteration
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
PPV_f and PPV_m is equal: False
['0', '1']
predic
['1', '1']
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
fpF: 1.0
fpM: 1.0
done create embeding vectors
['1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '0' '1']
['0', '1']
predic
['1', '1']
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
done iteration
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
PPV_f and PPV_m is equal: False
['0', '1']
predic
['1', '1']
[[0 1]
 [0 1]]
[

In [55]:
print(best_params, best_fpr_w,best_fpr_m, dataframe)

{'dimensions': 16, 'walk_length': 20, 'walk_number': 25, 'workers': 2} 1.0 1.0                            Female      Male
Group Fairness                0.0  0.000000
Predictive Parity             0.5  0.666667
False Positive Error Rate     1.0  1.000000
Equalized Opportunity         0.0  0.000000
Equalized Odds PPV            1.0  1.000000
Equalized Odds FPR            1.0  1.000000


In [59]:
# Parameter
parameter_grid = {
    'dimensions': [16, 32],
    'walk_length': [10, 15],
    'num_walks': [5, 7],
    'workers': [2, 4]
}

#apply Node2Vec embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_w,best_fpr_m, dataframe = perform_grid_search(nx_Graph, parameter_grid, list(newPublic.values()), list(newGender.values()))


Computing transition probabilities:   0%|          | 0/15 [00:00<?, ?it/s]

done create embeding vectors
['1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '0' '1']
['1', '1', '0', '1']
predic
['1', '1', '1', '1']
[[0 1]
 [0 3]]
[[0 0]
 [0 1]]
done iteration
[[0 1]
 [0 3]]
[[0 0]
 [0 1]]
PPV_f and PPV_m is equal: False
['1', '1', '0', '1']
predic
['1', '1', '1', '1']
[[0 1]
 [0 3]]
[[0 0]
 [0 1]]
[[0 1]
 [0 3]]
[[0 0]
 [0 1]]
[[0 1]
 [0 3]]
[[0 0]
 [0 1]]
fpF: 1.0
fpM: nan


C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\1293688598.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_m = fp_m/(tn_m + fp_m)
C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\1293688598.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_m = fp_m/(tn_m + fp_m)
C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\398579461.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_m = fp_m/(tn_m + fp_m)


Computing transition probabilities:   0%|          | 0/15 [00:00<?, ?it/s]

done create embeding vectors
['1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '0' '1']
['1', '1', '0']
predic
['1', '1', '1']
[[0 1]
 [0 2]]
[[0 0]
 [0 2]]
done iteration
[[0 1]
 [0 2]]
[[0 0]
 [0 2]]
PPV_f and PPV_m is equal: False
['1', '1', '0']
predic
['1', '1', '1']
[[0 1]
 [0 2]]
[[0 0]
 [0 2]]
[[0 1]
 [0 2]]
[[0 0]
 [0 2]]
[[0 1]
 [0 2]]
[[0 0]
 [0 2]]
fpF: 1.0
fpM: nan


...

done create embeding vectors
['1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '0' '1']
['1', '1', '0', '0']
predic
['1', '1', '1', '1']
[[0 2]
 [0 2]]
[[0 0]
 [0 1]]
done iteration
[[0 2]
 [0 2]]
[[0 0]
 [0 1]]
PPV_f and PPV_m is equal: False
['1', '1', '0', '0']
predic
['1', '1', '1', '1']
[[0 2]
 [0 2]]
[[0 0]
 [0 1]]
[[0 2]
 [0 2]]
[[0 0]
 [0 1]]
[[0 2]
 [0 2]]
[[0 0]
 [0 1]]
fpF: 1.0
fpM: nan


C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\1293688598.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_m = fp_m/(tn_m + fp_m)
C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\1293688598.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_m = fp_m/(tn_m + fp_m)
C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\398579461.py:17: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_m = fp_m/(tn_m + fp_m)


Computing transition probabilities:   0%|          | 0/15 [00:00<?, ?it/s]

done create embeding vectors
['1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '0' '1']
['1', '1', '1']
predic
['1', '1', '1']
[[0 0]
 [0 3]]
[[0 1]
 [0 1]]
done iteration
[[0 0]
 [0 3]]
[[0 1]
 [0 1]]
PPV_f and PPV_m is equal: False
['1', '1', '1']
predic
['1', '1', '1']
[[0 0]
 [0 3]]
[[0 1]
 [0 1]]
[[0 0]
 [0 3]]
[[0 1]
 [0 1]]
[[0 0]
 [0 3]]
[[0 1]
 [0 1]]
fpF: nan
fpM: 1.0


C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\1293688598.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_f = fp_f/(tn_f + fp_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\1293688598.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_f = fp_f/(tn_f + fp_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\398579461.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_f = fp_f/(tn_f + fp_f)


In [60]:
print(f"Best Parameters: {best_params}")
print(f"Best Fpr_w: {best_fpr_w}")
print(f"Best Fpr_m: {best_fpr_m}")

Best Parameters: {'dimensions': 16, 'num_walks': 5, 'walk_length': 15, 'workers': 2}
Best Fpr_w: 1.0
Best Fpr_m: 1.0


In [61]:
print(dataframe)

                           Female      Male
Group Fairness                0.0  0.000000
Predictive Parity             0.5  0.333333
False Positive Error Rate     1.0  1.000000
Equalized Opportunity         0.0  0.000000
Equalized Odds PPV            1.0  1.000000
Equalized Odds FPR            1.0  1.000000


In [66]:
# Parameter
parameter_grid = {
    'window': [16, 32],
    'walk_length': [20, 30],
    'number_of_random_walks': [25, 75],
}
#apply DeepWalk embedding, find the best embedding with the grid search algorithm
best_params, best_fpr_f, best_fpr_m,  dataframe =  perform_grid_search_DeepWalk(nx_Graph, parameter_grid, list(newPublic.values()), list(newGender.values()))

25
20
16


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 915.96it/s]

['1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '0' '1']


['1', '0']
predic
['1', '1']
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
fpF: 1.0
fpM: 1.0
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
PPV_f and PPV_m is equal: False
['1', '0']
predic
['1', '1']
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
[[0 1]
 [0 1]]
[[0 1]
 [0 2]]
25
20
32


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 833.11it/s]


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 476.85it/s]
C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\1293688598.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_f = fp_f/(tn_f + fp_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\1293688598.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_f = fp_f/(tn_f + fp_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\398579461.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_f = fp_f/(tn_f + fp_f)


['1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '0' '1']
['1', '1']
predic
['1', '1']
[[0 0]
 [0 2]]
[[0 1]
 [0 2]]
fpF: nan
fpM: 1.0
[[0 0]
 [0 2]]
[[0 1]
 [0 2]]
PPV_f and PPV_m is equal: False
['1', '1']
predic
['1', '1']
[[0 0]
 [0 2]]
[[0 1]
 [0 2]]
[[0 0]
 [0 2]]
[[0 1]
 [0 2]]
[[0 0]
 [0 2]]
[[0 1]
 [0 2]]
75
30
32


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 483.87it/s]


['1' '1' '0' '0' '1' '1' '0' '1' '1' '1' '1' '1' '1' '0' '1']
['1', '1', '1']
predic
['1', '1', '1']
[[0 0]
 [0 3]]
[[0 1]
 [0 1]]
fpF: nan
fpM: 1.0
[[0 0]
 [0 3]]
[[0 1]
 [0 1]]
PPV_f and PPV_m is equal: False
['1', '1', '1']
predic
['1', '1', '1']
[[0 0]
 [0 3]]
[[0 1]
 [0 1]]
[[0 0]
 [0 3]]
[[0 1]
 [0 1]]
[[0 0]
 [0 3]]
[[0 1]
 [0 1]]


C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\1293688598.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_f = fp_f/(tn_f + fp_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\1293688598.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_f = fp_f/(tn_f + fp_f)
C:\Users\nagel\AppData\Local\Temp\ipykernel_4088\398579461.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  fpr_f = fp_f/(tn_f + fp_f)


In [67]:
print(f"Best Parameters: {best_params}")
print(f"Best Fpr: {best_fpr_f}")
print(f"Best Fpr: {best_fpr_m}")
print(dataframe)

Best Parameters: {'number_of_random_walks': 25, 'walk_length': 20, 'window': 16}
Best Fpr: 1.0
Best Fpr: 1.0
                           Female      Male
Group Fairness                0.0  0.000000
Predictive Parity             0.5  0.666667
False Positive Error Rate     1.0  1.000000
Equalized Opportunity         0.0  0.000000
Equalized Odds PPV            1.0  1.000000
Equalized Odds FPR            1.0  1.000000
